In [93]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("/home/sramon/fotbal_data.csv")


In [94]:
def c_elo(df):
    elo = pd.DataFrame(np.zeros((len(df["Date"].unique())+1,len(df["AID"].unique()) )))
    i=0
    for team in df["HID"].unique():
        elo.rename(columns={i: f'{team}'}, inplace = True)
        i=i+1
    i=1
    for date in df["Date"].unique():
        elo.rename(index={i: f'{date}'}, inplace = True)
        i=i+1
    for team in elo:
        elo.at[0, f"{team}"] = 1500
    elo.rename(index={0 :"0"}, inplace = True)
    return elo

In [95]:
#aktualizace ela dvou týmu po zápase
def update_elo(index, K, home_team, away_team, elo,
                home_win, away_win, draw,today):
    mean_elo = 1500
    k_factor = 20
    draw_factor =0
    if draw==1:
        draw_factor = 0.5
   
    
    
    last_match_ofhome = elo[elo[f"{home_team}"] != 0].index[-1]
    last_match_ofaway = elo[elo[f"{away_team}"] != 0].index[-1]
  #  print(last_match_home)
    elo_home_team = elo.at[f"{last_match_ofhome}", f"{home_team}"]
    elo_away_team = elo.at[f"{last_match_ofaway}", f"{away_team}"]
    koeficient = (int(elo_away_team)-int(elo_home_team))/400
    liga = df.at[index, "LID"]
    expected_win_HOME = 1/(1+np.int32(10)**((int(elo_away_team)-int(elo_home_team))/400)) + K.at[f"{liga}","values"]
    expected_win_AWAY = 1/(1+np.int32(10)**((int(elo_home_team)-int(elo_away_team))/400)) - K.at[f"{liga}","values"]
    #print(expected_win_HOME, expected_win_AWAY)
    #print(elo_home_team, elo_away_team)
    elo_home = elo_home_team + k_factor * (home_win + draw_factor - expected_win_HOME)
    elo_away =elo_away_team +k_factor * (away_win + draw_factor - expected_win_AWAY)
   # print((k_factor * (home_win +draw_factor - expected_win_HOME)), k_factor * (away_win +draw_factor- expected_win_AWAY))
    return elo_home, elo_away

In [96]:
#výroba tabulky el row = týden, column = tým, postfixově k datu 
def create_elo(df, K):
    elo = c_elo(df)
    yesterday = "0"
    for index in df.index:
        today = df.at[index, "Date"]
        if(yesterday != today):
            for team in elo:
                elo.at[f"{today}",f"{team}"] = elo.at[f"{yesterday}" ,f"{team}"]
        yesterday = today
        home_team= df.at[index, "HID"]
        away_team= df.at[index, "AID"]
        home_win = df.at[index, "H"]
        draw =  df.at[index, "D"]
        away_win= df.at[index, "A"]
        elo.at[f'{today}', f'{home_team}'], elo.at[f'{today}', f'{away_team}'] = update_elo(index, K,home_team, away_team, elo,
                                                                                             home_win, away_win, draw,today)

    return elo    

In [109]:
def create_elo2(elo):
    i=1
    elo2 = elo
    for date in df["Date"].unique():  
        elo2.rename(index={f'{date}': i}, inplace = True)
        i=i+1
    elo2.rename(index={"0": 0}, inplace = True)

    i=0
    for date in df["Date"].unique():
        elo.rename(index={i : f'{date}'}, inplace = True)
        i=i+1
    return elo2

In [98]:
def expected_wins(K, home_team, away_team, df
                 ,index):
      
    
    elo_home_team = df.at[index, "elo_Home"]
    elo_away_team = df.at[index, "elo_Away"]
    liga = df.at[index, "LID"]
    koeficient = (int(elo_away_team)-int(elo_home_team))/400
    expected_win_HOME = 1/(1+np.int32(10)**((int(elo_away_team)-int(elo_home_team))/400)) +K.at[f"{liga}","values"]
    expected_win_AWAY = 1/(1+np.int32(10)**((int(elo_home_team)-int(elo_away_team))/400)) -K.at[f"{liga}","values"]
    return expected_win_HOME, expected_win_AWAY
    

In [99]:
def add_to_df(df, elo2):
    for index in df.index:
        today = df.at[index, "Date"]
        HID = df.at[index, "HID"]
        AID = df.at[index, "AID"]
        df.at[index, "elo_Home"] = elo2.at[f'{today}', f'{HID}']
        df.at[index, "elo_Away"] = elo2.at[f'{today}', f'{AID}']

In [100]:
def expected_r(K, df):
    expected = np.zeros((len(df), 2))
    for index in df.index:
        home_team = df.at[index, "elo_Home"]
        away_team = df.at[index, "elo_Away"]
        date = df.at[index, "Date"]
        expected_win_HOME, expected_win_AWAY = expected_wins(K, home_team, away_team,df,index)
    
        add = np.array([expected_win_HOME, expected_win_AWAY])
        
        expected[index, :] = add
    return expected

In [113]:
def compare(expected, j): 

    if (expected[0] - expected[1]) > j:
        return np.array([1, 0 ,0])

    elif (expected[1] - expected[0]) > j:
        return np.array([0, 0 ,1])
    else:
        return np.array([0, 1, 0])

In [102]:
def win_ratio_home(df, LD):
    liga = df[df["LID"] == f'{LD}']
    wins_home = len(liga[liga["H"] == 1])
    matches = len(liga)
    win_ratioH = wins_home/matches
    return win_ratioH
    

In [103]:
def draw_ratio(df, LD):
    liga = df[df["LID"] == f'{LD}']
    draws = len(liga[liga["D"] == 1])
    matches = len(liga)
    draws_ratios = draws/matches
    return draws_ratios
    

In [104]:
def win_ratio_away(df, LD):
    liga = df[df["LID"] == f'{LD}']
    wins_away = len(liga[liga["A"] == 1])
    matches = len(liga)
 #   print(wins_away)
#    print(matches)
    win_ratioA = wins_away/matches
    return win_ratioA

In [105]:
def ratio_per_LID(df):
   
    ratio = pd.DataFrame({'win_ratio_home':[0.0 for x in df["LID"].unique()],
                    'draw_ratio':[0.0 for x in df["LID"].unique()],
                    'win_ratio_away':[0.0 for x in df["LID"].unique()]},
                   index = [x for x in df["LID"].unique()])
    for liga in df["LID"].unique():
        ratio.at[f"{liga}", 'win_ratio_home'] = win_ratio_home(df, liga)
      #  print(win_ratio_home(df, liga))
        ratio.at[f"{liga}", 'draw_ratio'] = draw_ratio(df, liga)
        ratio.at[f"{liga}", 'win_ratio_away'] = win_ratio_away(df, liga)
        
    return ratio

In [110]:
ratio = ratio_per_LID(df)
K = pd.DataFrame({'values':[0.0 for x in df["LID"].unique()]},
                   index = [x for x in df["LID"].unique()])
for m in range(1,20):
    elo = create_elo(df, K)
    elo2 = create_elo2(elo)
    add_to_df(df, elo2)
    expected= expected_r(K, df)
    e = [x[0] for x in expected]
    df["expected_winsH"]= pd.DataFrame(e)
    e = [x[1] for x in expected]
    df["expected_winsA"]= pd.DataFrame(e)
    for liga in df["LID"].unique():
        e = df[df["LID"]==liga]
        s = ratio.at[f"{liga}","win_ratio_home"] + 0.5*ratio.at[f"{liga}","draw_ratio"] 
        K.at[f"{liga}","values"] = s - e["expected_winsH"].sum()/len(e)
        print(K.at[f"{liga}","values"])
        print(e["expected_winsH"].sum()/len(e))
        print("_____")
    print("______________________")

0.12849066381521695
0.4977416550874834
_____
0.106574770380159
0.497328373819335
_____
0.07905878758921375
0.49764575786533166
_____
0.08480645377436963
0.4979084133551492
_____
0.07912093015816546
0.4985046406180902
_____
0.05525044734998169
0.4986059356287417
_____
0.10228969035915675
0.49736065929119283
_____
0.05685402875803269
0.49885373379904496
_____
0.05754874244986402
0.49896055654870536
_____
0.1408025661521372
0.4964457559955138
_____
0.07407589967785982
0.4975550826898984
_____
0.20854454721081306
0.4940315652013648
_____
0.1030309656472565
0.4967435552095643
_____
0.11127493055303184
0.49705840278030156
_____
0.045682923671265696
0.501936123947782
_____
0.1193913785456378
0.4963493621951029
_____
0.13070712303082765
0.49643923837555015
_____
0.12418340110365156
0.49763478071453027
_____
0.12184894463400159
0.49598959703266504
_____
0.0974502412482291
0.496848004365806
_____
0.10774277423302725
0.497267886747783
_____
0.10771751471259416
0.4972824852874058
_____
0.084262771

KeyboardInterrupt: 

In [76]:
ratio = ratio_per_LID(df)
ratio

,win_ratio_home,draw_ratio,win_ratio_away
E1,0.477497,0.297471,0.225032
Q1,0.463318,0.281171,0.255511
R1,0.444930,0.263549,0.291521
A1,0.439071,0.287288,0.273641
B1,0.454338,0.246575,0.299087
I1,0.428191,0.251330,0.320479
D1,0.479021,0.241259,0.279720
L1,0.432420,0.246575,0.321005
M1,0.409871,0.293276,0.296853
C1,0.509396,0.255705,0.234899


In [111]:
df

,Unnamed: 0,Sea,Date,LID,HID,AID,Open,OddsH,OddsD,OddsA,HSC,ASC,H,D,A,elo_Home,elo_Away,expected_winsH,expected_winsA
0,0,2000,2000-03-19,E1,593,341,2000-03-15,0.00,0.00,0.00,3,0,1,0,0,1500.000000,1500.000000,0.631858,0.368142
1,1,2000,2000-03-19,E1,528,117,2000-03-15,0.00,0.00,0.00,2,0,1,0,0,1500.000000,1500.000000,0.631858,0.368142
2,2,2000,2000-03-19,E1,486,491,2000-03-15,0.00,0.00,0.00,0,0,0,1,0,1500.000000,1500.000000,0.631858,0.368142
3,3,2000,2000-03-19,E1,251,122,2000-03-15,0.00,0.00,0.00,0,0,0,1,0,1500.000000,1500.000000,0.631858,0.368142
4,4,2000,2000-03-19,E1,483,599,2000-03-15,0.00,0.00,0.00,0,1,0,0,1,1500.000000,1500.000000,0.631858,0.368142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37327,37327,2011,2011-06-30,C1,224,474,2011-06-26,2.26,3.74,3.18,2,1,1,0,0,1524.423014,1548.791756,0.610237,0.389763
37328,37328,2011,2011-06-30,C1,184,133,2011-06-26,2.30,3.72,3.10,0,1,0,0,1,1517.091724,1630.004562,0.487603,0.512397
37329,37329,2011,2011-06-30,C1,134,111,2011-06-26,1.73,4.17,4.82,3,1,1,0,0,1486.529753,1508.500432,0.613103,0.386897
37330,37330,2011,2011-06-30,O1,69,398,2011-06-26,2.09,4.03,3.35,2,0,1,0,0,1513.455196,1547.392536,0.576492,0.423508


In [117]:
#for j in range(1, 20):
    j=0
    predict = np.zeros((len(df), 3))


    i=0
    expected = expected_r(K, df)
    for row in expected:
            predict[i, :] = compare(row, j)

            i=i+1
    i=0
    correct=0
    results = df[["H", "D", "A"]].to_numpy()
    i=0
    correct=0
    for rowr, rowp in zip(results, predict):
        if (rowr == rowp).all():
            correct +=1
        i +=1
   
    print(correct/i)

0.4357387763848709


In [107]:
df

,Unnamed: 0,Sea,Date,LID,HID,AID,Open,OddsH,OddsD,OddsA,HSC,ASC,H,D,A,elo_Home,elo_Away,expected_winsH,expected_winsA
0,0,2000,2000-03-19,E1,593,341,2000-03-15,0.00,0.00,0.00,3,0,1,0,0,1500.000000,1500.000000,0.500000,0.500000
1,1,2000,2000-03-19,E1,528,117,2000-03-15,0.00,0.00,0.00,2,0,1,0,0,1500.000000,1500.000000,0.500000,0.500000
2,2,2000,2000-03-19,E1,486,491,2000-03-15,0.00,0.00,0.00,0,0,0,1,0,1500.000000,1500.000000,0.500000,0.500000
3,3,2000,2000-03-19,E1,251,122,2000-03-15,0.00,0.00,0.00,0,0,0,1,0,1500.000000,1500.000000,0.500000,0.500000
4,4,2000,2000-03-19,E1,483,599,2000-03-15,0.00,0.00,0.00,0,1,0,0,1,1500.000000,1500.000000,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37327,37327,2011,2011-06-30,C1,224,474,2011-06-26,2.26,3.74,3.18,2,1,1,0,0,1525.575249,1549.968634,0.465516,0.534484
37328,37328,2011,2011-06-30,C1,184,133,2011-06-26,2.30,3.72,3.10,0,1,0,0,1,1511.662617,1631.250315,0.333861,0.666139
37329,37329,2011,2011-06-30,C1,134,111,2011-06-26,1.73,4.17,4.82,3,1,1,0,0,1487.096736,1513.153547,0.462653,0.537347
37330,37330,2011,2011-06-30,O1,69,398,2011-06-26,2.09,4.03,3.35,2,0,1,0,0,1511.020790,1547.347336,0.448376,0.551624
